# Ensemble Learning Regressor

## Mục tiêu

- Xây dựng mô hình Ensemble Learning kết hợp các base learners đã huấn luyện từ các notebooks trước
- Sử dụng Voting Ensemble với weighted average (trọng số dựa trên MAE score)
- Đánh giá hiệu suất trên tập train bằng: MAE, RMSE, R²
- Lưu mô hình Ensemble

## Giới thiệu

Voting Ensemble kết hợp dự đoán của nhiều mô hình bằng weighted average:
- Giảm variance (lỗi ngẫu nhiên)
- Giảm bias (lỗi hệ thống)
- Tăng độ ổn định và tổng quát hóa

## Bước 1 - Import các thư viện cần thiết

In [14]:
from pathlib import Path
import joblib
import os
import pandas as pd
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, r2_score
import numpy as np


## Bước 2 - Đọc dữ liệu đã tiền xử lý

In [15]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")
print("\n5 dòng đầu tiên của tập train:")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3255, 15)

5 dòng đầu tiên của tập train:


,country_name,country_code,year,population,poverty_ratio,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,slum_population,labor_force
0,Romania,ROU,2019,-0.100387,-0.593216,-1.101137,75.607317,-0.107887,0.079336,1.142611,0.621932,0.746494,-0.047183,-0.857182,-0.871164
1,Mauritius,MUS,2006,-0.235882,-0.685495,-0.500838,72.432195,-0.405891,0.241659,0.204355,0.586683,0.718065,-0.221970,1.767314,-0.189257
2,Angola,AGO,2008,-0.083899,0.283435,1.500859,55.281000,-0.470953,1.327705,-0.839543,-1.545881,-2.268636,-0.432669,0.305989,1.598947
3,Albania,ALB,2001,-0.222240,-0.619770,-1.349940,75.083000,-0.584364,0.930412,-0.835544,0.600783,-0.024271,-0.404910,0.108810,-0.104497
4,Central African Republic,CAF,2021,-0.206911,2.567341,0.236338,40.279000,-0.618434,-0.427187,-1.336901,-2.349558,-3.008624,-0.529029,1.729562,1.172890


## Bước 3 - Chuẩn bị dữ liệu

In [16]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}") 
print(f"Kích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 12
Kích thước X_train: (3255, 12)
Kích thước y_train: (3255,)


## Bước 4 - Load các mô hình cơ sở (Base Learners)

In [18]:
BASE_DIR = Path("../model")

# ===== 1. Linear Models =====
linear_reg = joblib.load(BASE_DIR / "1_linear_regression" / "linear.pkl")
ridge_reg  = joblib.load(BASE_DIR / "1_linear_regression" / "ridge.pkl")
lasso_reg  = joblib.load(BASE_DIR / "1_linear_regression" / "lasso.pkl")

# ===== 2. SVM =====
linear_svr = joblib.load(BASE_DIR / "2_SVM_regression" / "linear_svr.pkl")
svr_rbf    = joblib.load(BASE_DIR / "2_SVM_regression" / "svr_rbf.pkl")

# ===== 3. Decision Tree =====
decision_tree = joblib.load(BASE_DIR / "3_decision_tree" / "decision_tree.pkl")

# ===== 4. Random Forest =====
random_forest = joblib.load(BASE_DIR / "4_random_forest" / "random_forest.pkl.gz")

# ===== 5. GBM =====
gbm = joblib.load(BASE_DIR / "5_GBM" / "GBM.pkl.gz")

# ===== 6. LightGBM =====
lightgbm = joblib.load(BASE_DIR / "6_lightgbm" / "lightgbm.pkl")    

# ===== 7. XGBoost =====
xgboost = joblib.load(BASE_DIR / "7_xgboost" / "xgboost.pkl.gz")

# ===== 8. CatBoost =====
catboost = joblib.load(BASE_DIR / "8_catboost" / "catboost.pkl.gz")

print("Tất cả các mô hình đã được load thành công!")

Tất cả các mô hình đã được load thành công!


## Bước 5 - Xây dựng Voting Ensemble với Weighted Average

### 5.1. Gom các model vào dictionary

In [19]:
# Gom models vào dictionary
base_models = {
    'Linear': linear_reg,
    'Ridge': ridge_reg,
    'Lasso': lasso_reg,
    'LinearSVR': linear_svr,
    'SVR_RBF': svr_rbf,
    'DecisionTree': decision_tree,
    'RandomForest': random_forest,
    'GBM': gbm,
    'XGBoost': xgboost,
    'LightGBM': lightgbm,
    'CatBoost': catboost
}

### 5.2. Tính toán R², MAE, RMSE của từng model

In [20]:
# Tính metrics cho từng model
print("="*70)
print("ĐÁNH GIÁ CÁC BASE LEARNERS TRÊN TẬP TRAIN")
print("="*70)

model_scores = {}

for model_name, model in base_models.items():
    y_pred = model.predict(X_train)
    mae = mean_absolute_error(y_train, y_pred)
    rmse = root_mean_squared_error(y_train, y_pred)
    r2 = r2_score(y_train, y_pred)
    
    model_scores[model_name] = {'MAE': mae, 'RMSE': rmse, 'R2': r2}
    
    print(f"\n{model_name:<15} | MAE: {mae:7.4f} | RMSE: {rmse:7.4f} | R²: {r2:7.4f}")

print("\n" + "="*70)

ĐÁNH GIÁ CÁC BASE LEARNERS TRÊN TẬP TRAIN

Linear          | MAE:  3.1268 | RMSE:  4.1540 | R²:  0.7706

Ridge           | MAE:  3.1278 | RMSE:  4.1541 | R²:  0.7705

Lasso           | MAE:  3.1273 | RMSE:  4.1544 | R²:  0.7705

LinearSVR       | MAE:  3.1636 | RMSE:  4.3065 | R²:  0.7534

SVR_RBF         | MAE:  1.4951 | RMSE:  2.5224 | R²:  0.9154

DecisionTree    | MAE:  1.5536 | RMSE:  2.2099 | R²:  0.9351

RandomForest    | MAE:  0.1984 | RMSE:  0.3446 | R²:  0.9984

GBM             | MAE:  0.1592 | RMSE:  0.2180 | R²:  0.9994

XGBoost         | MAE:  0.1002 | RMSE:  0.1330 | R²:  0.9998

LightGBM        | MAE:  0.2811 | RMSE:  0.3949 | R²:  0.9979

CatBoost        | MAE:  0.1955 | RMSE:  0.2634 | R²:  0.9991



### 5.2. Tính trọng số và Weighted Average Prediction

Tại bước này, ta sẽ áp dụng MAE để đánh trọng số cho các base learners.

1. MAE (Mean Absolute Error) được lựa chọn vì những lý do sau:

- MAE ổn định, ít nhạy cảm với outliers.
   - Dữ liệu được sử dụng có sự chênh lệch lớn về tuổi thọ giữa các quốc gia top đầu và các quốc gia top dưới. Do đó việc áp dụng MAE là phù hợp.

- MAE đo sai số trực tiếp theo đơn vị tuổi thọ.
   - Rất dễ diễn giải, trực quan, phù hợp với bài toán sức khỏe dân số.
   - Ví dụ: MAE = 1.8 tức là mô hình sai trung bình 1.8 năm tuổi thọ.

2. Các metrics còn lại (R2, RMSE) đều không phù hợp do:
- MAE không phản ánh sai số dự đoán thật, chỉ có thể giải thích phương sai của phân phối.
- RMSE cực nhạy với các outliers, có thể tăng mạnh đối với dữ liệu tuổi thọ hiện có.


3. Trọng số nghịch đảo với MAE:
   $$w_i = \frac{1/MAE_i}{\sum_j (1/MAE_j)}$$
   - MAE nhỏ → 1/MAE lớn → trọng số cao (mô hình tốt hơn)
   - MAE lớn → 1/MAE nhỏ → trọng số thấp (mô hình kém hơn)
   - Tự động chuẩn hóa: $\sum w_i = 1$

In [22]:
# Tính trọng số dựa trên MAE (nghịch đảo)
mae_values = {name: scores['MAE'] for name, scores in model_scores.items()}
inverse_mae = {name: 1.0 / mae for name, mae in mae_values.items()}
total_inverse_mae = sum(inverse_mae.values())
weights = {name: inv_mae / total_inverse_mae for name, inv_mae in inverse_mae.items()}

print("TRỌNG SỐ CỦA MỖI MODEL (dựa trên MAE nghịch đảo)")
print("="*70)
print(f"{'Model':<15} | {'MAE':<10} | {'1/MAE':<10} | {'Trọng số':<10} | {'%':<8}") 
print("-"*70)

for model_name in sorted(weights.keys(), key=lambda x: weights[x], reverse=True):
    mae = mae_values[model_name]
    inv = inverse_mae[model_name]
    w = weights[model_name]
    pct = w * 100
    print(f"{model_name:<15} | {mae:<10.4f} | {inv:<10.4f} | {w:<10.4f} | {pct:<8.2f}%")

TRỌNG SỐ CỦA MỖI MODEL (dựa trên MAE nghịch đảo)
Model           | MAE        | 1/MAE      | Trọng số   | %       
----------------------------------------------------------------------
XGBoost         | 0.1002     | 9.9794     | 0.3065     | 30.65   %
GBM             | 0.1592     | 6.2833     | 0.1930     | 19.30   %
CatBoost        | 0.1955     | 5.1149     | 0.1571     | 15.71   %
RandomForest    | 0.1984     | 5.0409     | 0.1548     | 15.48   %
LightGBM        | 0.2811     | 3.5575     | 0.1092     | 10.92   %
SVR_RBF         | 1.4951     | 0.6689     | 0.0205     | 2.05    %
DecisionTree    | 1.5536     | 0.6437     | 0.0198     | 1.98    %
Linear          | 3.1268     | 0.3198     | 0.0098     | 0.98    %
Lasso           | 3.1273     | 0.3198     | 0.0098     | 0.98    %
Ridge           | 3.1278     | 0.3197     | 0.0098     | 0.98    %
LinearSVR       | 3.1636     | 0.3161     | 0.0097     | 0.97    %


In [24]:
# Tính Weighted Average Prediction
y_pred_voting = np.zeros(len(y_train))

for model_name, weight in weights.items():
    model = base_models[model_name]
    y_pred_model = model.predict(X_train)
    y_pred_voting += weight * y_pred_model

print("Tính toán Weighted Average Prediction hoàn tất")

Tính toán Weighted Average Prediction hoàn tất


### 5.3. Đánh giá Voting Ensemble

In [27]:
# Tính metrics cho Voting Ensemble
mae_voting = mean_absolute_error(y_train, y_pred_voting)
rmse_voting = root_mean_squared_error(y_train, y_pred_voting)
r2_voting = r2_score(y_train, y_pred_voting)

print("KẾT QUẢ VOTING ENSEMBLE")
print("="*70)
print(f"MAE  : {mae_voting:7.4f}")
print(f"RMSE : {rmse_voting:7.4f}")
print(f"R²   : {r2_voting:7.4f}\n")

print("SO SÁNH VỚI CÁC BASE LEARNERS")
print("="*70)
print(f"{'Model':<15} | {'MAE':<10} | {'RMSE':<10} | {'R²':<10}")
print("-"*70)

# In từng model
for model_name in sorted(model_scores.keys(), key=lambda x: model_scores[x]['R2'], reverse=True):
    mae = model_scores[model_name]['MAE']
    rmse = model_scores[model_name]['RMSE']
    r2 = model_scores[model_name]['R2']
    print(f"{model_name:<15} | {mae:<10.4f} | {rmse:<10.4f} | {r2:<10.4f}")

print("-"*70)
print(f"{'VOTING':<15} | {mae_voting:<10.4f} | {rmse_voting:<10.4f} | {r2_voting:<10.4f}")


KẾT QUẢ VOTING ENSEMBLE
MAE  :  0.2548
RMSE :  0.3522
R²   :  0.9984

SO SÁNH VỚI CÁC BASE LEARNERS
Model           | MAE        | RMSE       | R²        
----------------------------------------------------------------------
XGBoost         | 0.1002     | 0.1330     | 0.9998    
GBM             | 0.1592     | 0.2180     | 0.9994    
CatBoost        | 0.1955     | 0.2634     | 0.9991    
RandomForest    | 0.1984     | 0.3446     | 0.9984    
LightGBM        | 0.2811     | 0.3949     | 0.9979    
DecisionTree    | 1.5536     | 2.2099     | 0.9351    
SVR_RBF         | 1.4951     | 2.5224     | 0.9154    
Linear          | 3.1268     | 4.1540     | 0.7706    
Ridge           | 3.1278     | 4.1541     | 0.7705    
Lasso           | 3.1273     | 4.1544     | 0.7705    
LinearSVR       | 3.1636     | 4.3065     | 0.7534    
----------------------------------------------------------------------
VOTING          | 0.2548     | 0.3522     | 0.9984    


## Bước 6 - Lưu Voting Ensemble Model

In [ ]:
# Tạo dictionary lưu Voting Ensemble
voting_ensemble_data = {
    'base_models': base_models,
    'weights': weights,
    'model_scores': model_scores,
    'metrics': {
        'MAE': mae_voting,
        'RMSE': rmse_voting,
        'R2': r2_voting
    },
    'feature_names': feature_cols
}

# Lưu mô hình
os.makedirs('../models/9_voting_ensemble', exist_ok=True)
voting_path = '../models/9_voting_ensemble/voting_ensemble.pkl.gz'

joblib.dump(voting_ensemble_data, voting_path, compress=('gzip', 9))

['../model/9_voting_ensemble/voting_ensemble.pkl.gz']

## Kết luận

### Tổng kết:
1. Mô hình Ensemble Learning đã được xây dựng thành công.
2. Sử dụng Voting Ensemble với weighted average (trọng số dựa trên MAE score)
3. Mô hình được đánh giá dựa trên 3 metrics trên tập train: MAE, RMSE và R2
